In [2]:
import os
import time
import math
import torch
import numpy as np
from torch import nn
from torch.autograd import Variable
from torchsummary import summary

In [3]:
%cd /home/kaixin1/Project/RTM/src/lib/models/networks
# %cd /home/kaixin1/Project/RTM/src

/home/kaixin1/Project/RTM/src/lib/models/networks


In [4]:
torch.__version__

'0.4.1.post2'

In [5]:
import resnet_FP
import imp
imp.reload(resnet_FP)

<module 'resnet_FP' from '/home/kaixin1/Project/RTM/src/lib/models/networks/resnet_FP.py'>

In [6]:
heads =  {'hm': 3, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2}
model = resnet_FP.get_FP_net(18,heads)

=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth
=> init deconv weights from normal distribution


In [10]:
import resnet_dcn
import imp
imp.reload(resnet_dcn)
model = resnet_dcn.get_pose_net(18,heads)
torch.cuda.set_device(1)
summary(model.cuda(1), (3, 384//4, 1280//4))

=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth
=> init deconv weights from normal distribution
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 48, 160]           9,408
       BatchNorm2d-2          [-1, 64, 48, 160]             128
              ReLU-3          [-1, 64, 48, 160]               0
         MaxPool2d-4           [-1, 64, 24, 80]               0
            Conv2d-5           [-1, 64, 24, 80]          36,864
       BatchNorm2d-6           [-1, 64, 24, 80]             128
              ReLU-7           [-1, 64, 24, 80]               0
            Conv2d-8           [-1, 64, 24, 80]          36,864
       BatchNorm2d-9           [-1, 64, 24, 80]             128
             ReLU-10           [-1, 64, 24, 80]               0
       BasicBlock-11           [-1, 64, 24, 80]               0
           Conv2d-12           [-

In [10]:
torch.cuda.set_device(3)
summary(model.cuda(3), (3, 384//4, 1280//4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 64, 48, 160]           9,408
       BatchNorm2d-2          [-1, 64, 48, 160]             128
              ReLU-3          [-1, 64, 48, 160]               0
         MaxPool2d-4           [-1, 64, 24, 80]               0
            Conv2d-5           [-1, 64, 24, 80]          36,864
       BatchNorm2d-6           [-1, 64, 24, 80]             128
              ReLU-7           [-1, 64, 24, 80]               0
            Conv2d-8           [-1, 64, 24, 80]          36,864
       BatchNorm2d-9           [-1, 64, 24, 80]             128
             ReLU-10           [-1, 64, 24, 80]               0
       BasicBlock-11           [-1, 64, 24, 80]               0
           Conv2d-12           [-1, 64, 24, 80]          36,864
      BatchNorm2d-13           [-1, 64, 24, 80]             128
             ReLU-14           [-1, 64,

/home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

In [6]:
args = ['ddd', '--exp_id', '3dop', '--dataset', 'kitti',\
'--kitti_split', '3dop' ,'--batch_size' ,'16' ,'--master_batch', '7' ,'--num_epochs','70',\
'--lr_step', '45,60' ,'--gpus', '1,2','--arch', 'resFP_18','--resume']

In [7]:
opt.debug

NameError: name 'opt' is not defined

In [8]:
opt = opts().parse(args)

opt.gpus 1,2
opt.gpus 1,2
opt.gpus [1, 2]
Fix size testing.
training chunk_sizes: [7, 9]
The output will be saved to  /home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop


In [9]:
opt.gpus

[1, 2]

In [10]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test
# Dataset多继承了数据库类(KITTI)和任务类(ddd)；
Dataset = get_dataset(opt.dataset, opt.task)
# 在参数里添加与用到的数据集的相关属性
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)

heads {'hm': 3, 'vertex_hm': 9, 'vertex_coordinate': 18, 'dep': 1, 'rot': 8, 'dim': 3, 'wh': 2, 'reg': 2, 'vertex_reg_offset': 2}


In [11]:
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str

In [12]:
print('Creating model...')
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(\
  model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

Creating model...
=> loading pretrained model https://download.pytorch.org/models/resnet18-5c106cde.pth
=> init deconv weights from normal distribution
loaded /home/kaixin1/Project/RTM/src/lib/../../exp/ddd/3dop/model_last.pth, epoch 10
Resumed optimizer with start lr 0.000125


In [13]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)

In [15]:
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

In [16]:
trainer.set_device(opt.gpus, opt.chunk_sizes,opt.device )

In [17]:
print('Setting up data...')
val_loader = torch.utils.data.DataLoader(
  Dataset(opt, 'val'), 
  batch_size=1, 
  shuffle=False,
  num_workers=1,
  pin_memory=True
)

train_loader = torch.utils.data.DataLoader(
  Dataset(opt, 'train'), 
  batch_size=opt.batch_size, 
  shuffle=True,
  num_workers=opt.num_workers,
  pin_memory=True,
  drop_last=True
)

Setting up data...
==> initializing kitti 3dop, val data.
loading annotations into memory...
Done (t=1.00s)
creating index...
index created!
Loaded val 3769 samples
==> initializing kitti 3dop, train data.
loading annotations into memory...
Done (t=1.00s)
creating index...
index created!
Loaded train 3712 samples


In [18]:
print('Starting training...')
best = 1e10
for epoch in range(start_epoch + 1, opt.num_epochs + 1):
    mark = epoch if opt.save_all else 'last'
    print("go epoch:",epoch)
    log_dict_train, _ = trainer.train(epoch, train_loader)
    print("go out",epoch)
    logger.write('epoch: {} |'.format(epoch))
    for k, v in log_dict_train.items():
      logger.scalar_summary('train_{}'.format(k), v, epoch)
      logger.write('{} {:8f} | '.format(k, v))
    if opt.val_intervals > 0 and epoch % opt.val_intervals == 0:
      save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(mark)), 
                 epoch, model, optimizer)
      with torch.no_grad():
        log_dict_val, preds = trainer.val(epoch, val_loader)
      for k, v in log_dict_val.items():
        logger.scalar_summary('val_{}'.format(k), v, epoch)
        logger.write('{} {:8f} | '.format(k, v))
      if log_dict_val[opt.metric] < best:
        best = log_dict_val[opt.metric]
        save_model(os.path.join(opt.save_dir, 'model_best.pth'), 
                   epoch, model)
    else:
      save_model(os.path.join(opt.save_dir, 'model_last.pth'), 
                 epoch, model, optimizer)
    logger.write('\n')
    if epoch in opt.lr_step:
      save_model(os.path.join(opt.save_dir, 'model_{}.pth'.format(epoch)), 
                 epoch, model, optimizer)
      lr = opt.lr * (0.1 ** (opt.lr_step.index(epoch) + 1))
      print('Drop LR to', lr)
      for param_group in optimizer.param_groups:
          param_group['lr'] = lr
    logger.close()

Starting training...
go epoch: 11


RuntimeError: CUDA error (10): invalid device ordinal (check_status at /opt/conda/conda-bld/pytorch_1535491974311/work/aten/src/ATen/cuda/detail/CUDAHooks.cpp:36)
frame #0: torch::cuda::scatter(at::Tensor const&, at::ArrayRef<long>, at::optional<std::vector<long, std::allocator<long> > > const&, long, at::optional<std::vector<CUDAStreamInternals*, std::allocator<CUDAStreamInternals*> > > const&) + 0x4e1 (0x7f14da863871 in /home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/_C.cpython-36m-x86_64-linux-gnu.so)
frame #1: <unknown function> + 0xc42a0b (0x7f14da86ba0b in /home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/_C.cpython-36m-x86_64-linux-gnu.so)
frame #2: <unknown function> + 0x38a5cb (0x7f14d9fb35cb in /home/kaixin1/anaconda3/envs/CenterNet/lib/python3.6/site-packages/torch/_C.cpython-36m-x86_64-linux-gnu.so)
frame #3: _PyCFunction_FastCallDict + 0x154 (0x55962670d4f4 in /home/kaixin1/anaconda3/envs/CenterNet/bin/python)
frame #4: <unkn

In [ ]:
self = Dataset(opt, 'val')

In [ ]:
import torch.utils.data as data
import pycocotools.coco as coco
import numpy as np
import torch
import json
import cv2
import os
import math
from utils.image import flip, color_aug
from utils.image import get_affine_transform, affine_transform
from utils.image import gaussian_radius, draw_umich_gaussian, draw_msra_gaussian
import pycocotools.coco as coco

In [ ]:
index = 0
img_id = self.images[index]
img_info = self.coco.loadImgs(ids=[img_id])[0]
img_path = os.path.join(self.img_dir, img_info['file_name'])
img = cv2.imread(img_path)
if 'calib' in img_info:
  calib = np.array(img_info['calib'], dtype=np.float32)
else:
  calib = self.calib

In [ ]:
height, width = img.shape[0], img.shape[1]
c = np.array([img.shape[1] / 2., img.shape[0] / 2.])
if self.opt.keep_res:
  s = np.array([self.opt.input_w, self.opt.input_h], dtype=np.int32)
else:
  s = np.array([width, height], dtype=np.int32)

aug = False
if self.split == 'train' and np.random.random() < self.opt.aug_ddd:
  aug = True
  sf = self.opt.scale
  cf = self.opt.shift
  s = s * np.clip(np.random.randn()*sf + 1, 1 - sf, 1 + sf)
  c[0] += img.shape[1] * np.clip(np.random.randn()*cf, -2*cf, 2*cf)
  c[1] += img.shape[0] * np.clip(np.random.randn()*cf, -2*cf, 2*cf)

trans_input = get_affine_transform(
  c, s, 0, [self.opt.input_w, self.opt.input_h])
inp = cv2.warpAffine(img, trans_input, 
                     (self.opt.input_w, self.opt.input_h),
                     flags=cv2.INTER_LINEAR)
inp = (inp.astype(np.float32) / 255.)
# if self.split == 'train' and not self.opt.no_color_aug:
#   color_aug(self._data_rng, inp, self._eig_val, self._eig_vec)
inp = (inp - self.mean) / self.std
inp = inp.transpose(2, 0, 1)

num_classes = self.opt.num_classes
trans_output = get_affine_transform(
  c, s, 0, [self.opt.output_w, self.opt.output_h])

hm = np.zeros(
  (num_classes, self.opt.output_h, self.opt.output_w), dtype=np.float32)
wh = np.zeros((self.max_objs, 2), dtype=np.float32)
reg = np.zeros((self.max_objs, 2), dtype=np.float32)
dep = np.zeros((self.max_objs, 1), dtype=np.float32)
rotbin = np.zeros((self.max_objs, 2), dtype=np.int64)
rotres = np.zeros((self.max_objs, 2), dtype=np.float32)
dim = np.zeros((self.max_objs, 3), dtype=np.float32)
ind = np.zeros((self.max_objs), dtype=np.int64)
reg_mask = np.zeros((self.max_objs), dtype=np.uint8)
rot_mask = np.zeros((self.max_objs), dtype=np.uint8)

ann_ids = self.coco.getAnnIds(imgIds=[img_id])
anns = self.coco.loadAnns(ids=ann_ids)
num_objs = min(len(anns), self.max_objs)
draw_gaussian = draw_msra_gaussian if self.opt.mse_loss else \
                draw_umich_gaussian
gt_det = []


In [ ]:
??draw_gaussian

In [ ]:
for k in range(num_objs):
      ann = anns[k]
      bbox = self._coco_box_to_bbox(ann['bbox'])
      cls_id = int(self.cat_ids[ann['category_id']])
      if cls_id <= -99:
        continue
      # if flipped:
      #   bbox[[0, 2]] = width - bbox[[2, 0]] - 1
      bbox[:2] = affine_transform(bbox[:2], trans_output)
      bbox[2:] = affine_transform(bbox[2:], trans_output)
      bbox[[0, 2]] = np.clip(bbox[[0, 2]], 0, self.opt.output_w - 1)
      bbox[[1, 3]] = np.clip(bbox[[1, 3]], 0, self.opt.output_h - 1)
      h, w = bbox[3] - bbox[1], bbox[2] - bbox[0]
      if h > 0 and w > 0:
        radius = gaussian_radius((h, w))
        radius = max(0, int(radius))
        ct = np.array(
          [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2], dtype=np.float32)
        ct_int = ct.astype(np.int32)
        if cls_id < 0:
          ignore_id = [_ for _ in range(num_classes)] \
                      if cls_id == - 1 else  [- cls_id - 2]
          print("k:{} ,ignore_id:{}".format(k,ignore_id))
          if self.opt.rect_mask:
            hm[ignore_id, int(bbox[1]): int(bbox[3]) + 1, 
              int(bbox[0]): int(bbox[2]) + 1] = 0.9999
          else:
            for cc in ignore_id:
              draw_gaussian(hm[cc], ct, radius)
            hm[ignore_id, ct_int[1], ct_int[0]] = 0.9999
          continue
        print("k:{} ,cls_id:{}".format(k,cls_id))
#         draw_gaussian(hm[cls_id], ct, radius)
        print("ct:{},radius:{}".format(ct,radius))
        heatmap, center, radius = hm[cls_id], ct, radius
        diameter = 2 * radius + 1
        gaussian = gaussian2D((diameter, diameter), sigma=diameter / 6)
        x, y = int(center[0]), int(center[1])

        height, width = heatmap.shape[0:2]

        left, right = min(x, radius), min(width - x, radius + 1)
        top, bottom = min(y, radius), min(height - y, radius + 1)
        print(left, right,top, bottom)
        masked_heatmap  = heatmap[y - top:y + bottom, x - left:x + right]
        print("masked_heatmap",masked_heatmap)
        masked_gaussian = gaussian[radius - top:radius + bottom, radius - left:radius + right]
        print("masked_gaussian",masked_gaussian)
        if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0: # TODO debug
            np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)

In [ ]:
_, preds = trainer.val(0, val_loader)
val_loader.dataset.run_eval(preds, opt.save_dir)

In [25]:
from math import cos,sin
ry3d = 1.57
K=np.array([[ 721.53, 0, 609.559,0], [0, 721.537,172.854,0],[0, 0, 1,0],[0,0,0,1]])
cx3d, cy3d, cz = -16.53 ,2.39, 58.49
T = np.array([[+cos(ry3d), 0, +sin(ry3d),cx3d],
          [0, 1, 0,cy3d],
          [-sin(ry3d), 0, cos(ry3d),cz],
             [0,0,0,1]])

In [36]:
w = 1.68
h = 1.9
l = 4.0
x_corners = [l/2, l/2, -l/2, -l/2, l/2, l/2, -l/2, -l/2,0]
y_corners = [0,0,0,0,-h,-h,-h,-h,-h/2]
z_corners = [w/2, -w/2, -w/2, w/2, w/2, -w/2, -w/2, w/2,0]

linear_item = [1,1,1,1,1,1,1,1,1]

In [37]:
P3d =np.vstack([np.array(x_corners),np.array(y_corners),np.array(z_corners),np.array(linear_item)])

In [38]:
P3d

array([[ 2.  ,  2.  , -2.  , -2.  ,  2.  ,  2.  , -2.  , -2.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  ,  0.  , -1.9 , -1.9 , -1.9 , -1.9 , -0.95],
       [ 0.84, -0.84, -0.84,  0.84,  0.84, -0.84, -0.84,  0.84,  0.  ],
       [ 1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ,  1.  ]])

In [41]:
project = K.dot(T).dot(P3d)

In [43]:
project[:2]/project[2]

array([[409.17800194, 387.71486804, 402.34683611, 422.39037778,
        409.17800194, 387.71486804, 402.34683611, 422.39037778,
        405.6456661 ],
       [203.38069483, 203.38141779, 201.36272075, 201.36209024,
        179.11261107, 179.1127593 , 178.69888417, 178.6987549 ,
        190.61794734]])